In [ ]:
# -*- coding: utf-8 -*-
"""NSGA3-IoT.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1tqMznLwdA1XOKOWky77OQSAw1bGIS9IH
"""

# Commented out IPython magic to ensure Python compatibility.
from itertools import combinations
# -*- coding: utf-8 -*-
""" NSGA-III """

# Commented out IPython magic to ensure Python compatibility.
# %pip -q install deap
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use backend 'Agg' to prevent displaying plots
from deap import base, creator, tools, algorithms
from deap.tools.indicator import hv
from functools import partial
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
import time
import random
import json
import os
#from google.colab import drive

#---------------- Clear terminal window before program execution ---------------
os.system('cls' if os.name == 'nt' else 'clear')
# On Windows (nt) systems, use the cls command
# On Unix/Linux and MacOS systems, use the clear command
#-------------------------------------------------------------------------------
# -------------------- Infinite value for int ----------------------------------
INF = 10**9  # Infinite Constant for int
# ------------------------------------------------------------------------------

# ------------------- Function Declarations ------------------------------------


# NSGA3 Hyperparameter Default Values
default_params = {
  'population_size': 1500,
  'generations': 80,
  'cx_prob': 0.4,
  'indpb': 0.9
}

# Function to read parameters from JSON file
def read_hyper_parameters(directory):
    file_name = 'hyper_parameters.json'
    file_path = os.path.join(directory, file_name)

    print(f"Checking if file {file_name} exists in path: {file_path}")

    if os.path.exists(file_path):
        print(f"File {file_name} found. Trying to read parameters.")
        with open(file_path, 'r') as file:
            try:
                params = json.load(file)
                print(f"Parameters read from file: {params}")
                return params
            except json.JSONDecodeError as e:
                print(f"Error reading JSON file: {e}")
                return default_params  # If there is an error, return the default values
    else:
        print(f"File {file_name} not found in directory {directory}. Using default values.")
        return default_params  # If the file does not exist, use the default values

def mark_busy(cont_nos_rede, v_nodes_network, V_Busy):
    pos = 0  # Specifies the node nº in the network
    for i in range(cont_nos_rede):
        pos = v_nodes_network[i]
        V_Busy[pos] = 1  # Assigns the node at position i as occupied.
    return V_Busy


# Function to check if solution A dominates solution B
def dominates(A, B):
    return all(a <= b for a, b in zip(A, B)) and any(a < b for a, b in zip(A, B))

# Helper function to calculate accumulated latency from a node
def calculate_latency(mst, start_node, end_node, visited):
    if start_node == end_node:
        return 0

    visited[start_node] = True
    latency = INF #float('inf')

    for i in range(len(mst)):
        u = mst[i][0]
        v = mst[i][1]
        w = mst[i][2]

        if u == start_node and not visited[v]:
            latency = min(latency, w + calculate_latency(mst, v, end_node, visited))
        elif v == start_node and not visited[u]:
            latency = min(latency, w + calculate_latency(mst, u, end_node, visited))

    return latency

# Helper function to find the component representative
def findComponent(components, node):
    return components.index(components[node]) + 1  # Adjust for 0-indexing

# Function to remove edges surrounding the inactive node
def remove_edges_involving_node(adjList, inactive_node):
    for i in range(len(adjList)):
        if i != inactive_node:
            adjList[i] = [connection for connection in adjList[i] if connection[0] != inactive_node]

# Define the objective function
def objectiveFunction(individual, aux, job, jr, jb, N_R, N_B, N_L, v_sol_f1, v_sol_f2, v_sol_f3, v_sol_fx, v_sol_ind, num):

    # Initializing the value of the lists of objective function components
    f = np.zeros(3)
    # Initializing value of the objective function
    OF = 0

    f[0] = (np.sum(N_R[:num] * individual[:num])) - jr[job] # Minimize
    f[1] = (np.sum(N_B[:num] * individual[:num])) - jb[job] # Minimize
    f[2] = aux - (np.sum(N_L[:num] * individual[:num])) # Maximize

    if (f[0] >= 0 and f[1] >= 0 and f[2] >= 0):
        # The calculation of OF considers the attributes: R, B and L
        OF = f[0]**2 + f[1]**2 - f[2]
        #----------- Storing the solutions -----------------------
        v_sol_f1.append(f[0])
        v_sol_f2.append(f[1])
        v_sol_f3.append(f[2])
        v_sol_fx.append(OF)
        v_sol_ind.append(individual)

    # Returns the values ​​of each of the objectives
    return f[0], f[1], f[2]

# Constraint function - returns values: c[0], c[1] e c[2] <= 0
def jobConstraints(individual, aux, job, jr, jb, N_R, N_B, N_L, num):

    c = np.zeros(3)
    c[0] = jr[job] - (np.sum(N_R[:num] * individual[:num]))
    c[1] = jb[job] - (np.sum(N_B[:num] * individual[:num]))
    c[2] = (np.sum(N_L[:num] * individual[:num])) - aux

    return c[0], c[1], c[2]

def get_positive_integer(prompt, default):
    while True:
        try:
            user_input = input(f"{prompt} ({default}): ")
            # If the user does not enter anything, it uses the default value.
            value = int(user_input) if user_input else default

            if value > 0:
                return value  # Returns the value if it is positive
            else:
                print("Erro: The number must be a positive value. Please try again.")
        except ValueError:  # Checks if value is an integer
            print("Erro: Invalid input. Please enter an integer.")

# -------------------------- End Functions -------------------------------------

# -------------------------- Main Function -------------------------------------
def main(path_input, path_output, numRunnings):

 #drive.mount('/content/drive')
 for r in range(numRunnings):

  input_file_name =  os.path.basename(path_input).split('.')[0]

  # Check if the output folder exists, if not, create it
  if not os.path.exists(path_output):
      os.makedirs(path_output)

  # Specify the path to save the file

  file_path = f'{path_output}/results_{str(r)}_{input_file_name}.txt'

  output_file_name = os.path.basename(file_path).split('.')[0]

  name_file = output_file_name + str(r) + '.txt'

  try:
    fileID = open(file_path, 'w')
  except IOError:
    raise Exception('Error creating or opening file for writing.')

  # Load input data from a JSON file
  with open(path_input, 'r') as f:
      data = json.load(f)

  fileID.write("\n----------------------------------------------")
  fileID.write("\n---------------- NSGA-III --------------------")
  fileID.write("\n----------------------------------------------")

  # ---------------------------- Job Input Parameters ----------------------------
  jr = np.array(data['jr'])
  jb = np.array(data['jb'])
  jl = np.array(data['jl'])
  jo = np.array(data['jo'])
  # ------------------------------------------------------------------------------
  n_jobs = len(jr)  # Nº of jobs
  # ------------------------------------------------------------------------------------------
  # t_c = time_connection = 1 (1 ms) is the connection time of any IoT application to the source node of the network
  t_c = 1
  aux = 0 # variable that stores the latency value of a job minus t_c
  # ------------------------------------------------------------------------------
  # Initial values ​​of vectors R and B
  N_R = np.array(data['V_R'])
  N_B = np.array(data['V_B'])
  #----------------- No. of IoT network nodes (Variables)------------------------------------
  numNodes = len(N_R)
  #-------------------------------------------------------------------------------
  # Initial node values ​​regarding the status: free/busy ------------------
  # The values ​​represent: 0 (free node) and 1 (busy node) ------------------------
  V_Busy = np.array(data['V_Busy'])

  #-------------------------------------------------------------------------------
  # Initial values ​​of the nodes regarding the state: Active/Inactive -------------
  # The values ​​represent: 0 (Active node) and 1 (Inactive node) ------------------
  V_Inactive = np.array(data['V_Inactive'])
  #-------------------------------------------------------------------------------
  # Counter of the total number of allocated nodes of the entire network
  cont_nos_rede = 0
  # Stores the indexes of the allocated nodes of the entire network
  v_nodes_network = []
  #--------- Representation of IoT network with adjacency list as a cell ---------
  #-------- IoT network representation with adjacency list as a cell -------------
  # Initializing adjacency lists as a list
  adjList = data['adjList']

  #---- Step 1: Convert the adjacency list to an edge list ----
  # Initializing the edge list
  edges = []

  # Making a given node inactive: replacing the adjacency list with a loop to itself and infinite latency
  for i in range(numNodes):
      if V_Inactive[i]: # 0 = False e 1 = True
        adjList[i] = [[i, INF]]
        # Remove surrounding edges from a node
        remove_edges_involving_node(adjList, i)

  # Printing the updated adjacency list
  #print("Lista de adjacências atualizada:")
  #for i, connections in enumerate(adjList):
      #print(f"Nó {i}: {connections}")

  # Traversing the adjacency list to build the edge list
  for i in range(len(adjList)):
      connections = adjList[i]
      for j in range(len(connections)):
          node = connections[j][0]
          latency = connections[j][1]
          # Adding non-duplicate edges
          if i < node:  # i because Python is 0-indexed
              edges.append([i, node, latency])

  # Displaying the list of edges
  #print('Edge List:')
  #print(np.array(edges))

  #---- Step 2: Apply Kruskal's Algorithm ----
  # Class to represent an edge in the graph
  mst = []
  class Edge:
      def __init__(self, u, v, weight):
          self.u = u  # Source node
          self.v = v  # Target node
          self.weight = weight  # Edge weight (latency)

  # Class to represent a subset for union-find
  class Subset:
      def __init__(self, parent, rank):
          self.parent = parent
          self.rank = rank

  # Find the representative of the set to which the node belongs
  def find(subsets, node):
      if subsets[node].parent != node:
          subsets[node].parent = find(subsets, subsets[node].parent)
      return subsets[node].parent

  # Perform the union of two sets
  def union(subsets, u_root, v_root):
      # Attach the shorter tree under the root of the taller tree
      if subsets[u_root].rank < subsets[v_root].rank:
          subsets[u_root].parent = v_root
      elif subsets[u_root].rank > subsets[v_root].rank:
          subsets[v_root].parent = u_root
      else:
          subsets[v_root].parent = u_root
          subsets[u_root].rank += 1

  # Main function to find MST using Kruskal's algorithm
  def kruskal_mst(adjList):
      edges = []

      # Convert adjacency list to edge list
      for u in range(len(adjList)):  # Fix for iterating over indexes
          for v, weight in adjList[u]:  # Keeps the structure, but now u is an index
              edges.append(Edge(u, v, weight))

      # Sort edges by weight
      edges = sorted(edges, key=lambda edge: edge.weight)

      # Create individual subsets for all nodes
      subsets = [Subset(i, 0) for i in range(len(adjList))]

      mst = []  # List to store MST edges

      # Iterate through the edges and build the MST
      for edge in edges:
          u_root = find(subsets, edge.u)
          v_root = find(subsets, edge.v)

          # If the edge does not form a cycle, add it to the MST
          if u_root != v_root:
              mst.append([edge.u, edge.v, edge.weight])
              union(subsets, u_root, v_root)

      return np.array(mst)
  #-------------------------------------------------------------------------------
  # Calculate MST
  mst = kruskal_mst(adjList)
  # Displaying the minimum spanning tree
  #print('Minimum Spanning Tree (MST):')
  #print(np.array(mst))
  #-------------------------------------------------------------------------------
  #--------------------- Initializing the AN_L list ------------------------------
  # Creating and initializing a list of lists, with all elements initially equal to zero
  AN_L = [[0 for _ in range(numNodes)] for _ in range(numNodes)]
  #-------------------------------------------------------------------------------
  edge_nodes = data['edge_nodes']  # Nodes that are part of the edge (source nodes)- user defined (set to zero index)
  size_edge_nodes = len(edge_nodes)

  for t in range(size_edge_nodes):
      node = edge_nodes[t]
      for i in range(numNodes):
          if i != node:
            AN_L[node][i] = calculate_latency(mst, node, i, np.zeros(numNodes, dtype=bool))

  # Displaying the list of accumulated AN_L latencies
  print('AN_L List (Accumulated latencies of nodes (edges nodes) in relation to the other nodes):')
  print(AN_L)
  #input('ENTER...')
  #-------------------------------------------------------------------------------
  # Upper and Lower bounds
  ub = np.ones(numNodes)
  lb = np.zeros(numNodes)
  #-------------- start_time, end_time, and total elapsed time -------------------
  start_time = 0
  end_time = 0
  total_time = 0
  #-------------------------------------------------------------------------------
  #------------------------------ Total OF ---------------------------------------
  OF_total = 0
  #-------------------------------------------------------------------------------
  # -------------------- Define the parameters NSGA-III --------------------------

  # Reading parameters from file or using default values
  hyper_parameters = read_hyper_parameters(input_folder)

  # Updating parameters in code
  population_size = hyper_parameters.get('population_size', default_params['population_size'])    # initial population on which the algorithm will operate

  generations = hyper_parameters.get('generations', default_params['generations'])    # number of generations (iterations) - defines how many times the selection, crossover, mutation and evaluation process will be repeated (default=200).

  cx_prob = hyper_parameters.get('cx_prob', default_params['cx_prob'])    # defines how many times the selection, crossover, mutation and evaluation process will be repeated - If it is 0.30, it means that 30% of the selected parent pairs will be crossed.

  mut_prob = (1/numNodes) # frequency with which the mutation will be applied to the generated descendants - If it is (1/numNodes), it means that (1/numNodes)% of the descendants will undergo mutation (default: 1% to 5%).

  indpb = hyper_parameters.get('indpb', default_params['indpb']) # In uniform crossover, each gene from two parents has a probability (usually 0.9 - equals 90%) of being exchanged, resulting in offspring with genes randomly inherited from either parent (default: 70% to 90%).

  majority = round((numNodes / 2) + 1) # simple majority = half + 1

  # Defina o Hall of Fame
  #hof = hall_of_fame = tools.HallOfFame(1)  # Stores the best individual
  # ------------------------------------------------------------------------------
  # Output value of the population generated by NSGA3
  #population = []
  # Output value of the population generated by NSGA3, without replication in the solutions
  #final_population_no_replicate = []
  # Get unique rows based on unique values' indices
  #final_population = []
  #-------------------------------------------------------------------------------
  # --------------------------- Configure DEAP -----------------------------------
  creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, 1.0)) # -1.0 = Minimize, 1.0 = Maximize, 0.0 = Without a specific optimization direction (free)
  creator.create("Individual", list, fitness=creator.FitnessMulti)
  #-------------------------------------------------------------------------------
  # The bounded_crossover function performs the crossovers using cxUniform and cxOnePoint, and ensures that the genes of the individuals remain within the interval [0,1)
  def bounded_crossover(ind1, ind2, majority, indpb):
      """
      Applies crossover to the genes of two individuals.

      Parameters:
      ind1 (list of int): The first individual.
      ind2 (list of int): The second individual.
      indpb (float): Mutation probability for each gene.
      """

      cont = 0 # counts how many elements of the tuple are different
      for elem1, elem2 in zip(ind1, ind2):
          if (elem1 != elem2):
              cont += 1

      # Modifies only randomly generated individuals, with amount of (elements<=majority) with an individual from the population
      if (cont <= majority) and (cont > 0):
        # cxTwoPoint e cxOnePoint: Swap entire segments of genes, ensuring that values ​​remain within range
        #tools.cxOnePoint(ind1, ind2)
        # The cxUniform operator performs crossover by randomly swapping the genes of the parents. This keeps the gene values ​​within the range [0,1) because the swapped genes are just copied from the parents.
        # cxUniform: Keeps values ​​within 0 or 1 because it only exchanges the genes from the parents.
        tools.cxUniform(ind1, ind2, indpb)

      return ind1, ind2

  #-------------------------------------------------------------------------------

  def bounded_mutation(individual, majority, numNodes):
      """
      Applies mutation to the genes of an individual.

      Parameters:
      individual (list of int): The individual to be mutated.
      indpb (float): Probability of mutation for each gene.

      Returns:
      individual (list of int): The mutated individual.
      """
      cont = 0 # counts how many elements of the tuple are different
      modify = []  # generates an element from an individual and modifies it randomly
      # Compares the randomly generated individual with an individual from the population
      for i in range(numNodes):
          modify.append(random.randint(0, 1))

          if individual[i] != modify[i]:
              cont += 1
      # Modifies only randomly generated individuals, with qde of (elements<=majority) with an individual from the population
      # Up to 50% of the number of elements in a tuple that make up an individual in the population
      if (cont <= majority) and (cont > 0):
          for i in range(numNodes):
            # Apply the mutation to the individual
              individual[i] = modify[i]

      return individual, # Returning the mutated individual as a tuple (1 element)
  #-------------------------------------------------------------------------------

  # Defining the attr_int function to generate int numbers: 0 or 1
  def attr_int():
      return random.randint(0, 1)

  toolbox = base.Toolbox()
  # ------------------------------------------------------------------------------
  # Registering the attr_int function in the toolbox, toolbox.attr_int() to generate numbers: 0 or 1
  toolbox.register("attr_int", attr_int)
  toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=numNodes)
  toolbox.register("population", tools.initRepeat, np.array, toolbox.individual)
  toolbox.register("mate", bounded_crossover, majority=majority, indpb=indpb)
  #-------------------------------------------------------------------------------
  # Each gene has a 50% chance of being mutated. If the gene is not chosen for mutation, it remains unchanged.
  toolbox.register("mutate", bounded_mutation, majority=majority, numNodes=numNodes)

  # 3 = Nº Objectives (R e B) and 30 = Number of divisions made in hyperspace to create the reference points
  toolbox.register("select", tools.selNSGA3, ref_points=tools.uniform_reference_points(3, 30))

  # Logbook and statistics configuration
  # ind: This is the parameter that represents the input argument to this anonymous function.
  # In the context of DEAP (a library of evolutionary algorithms), ind is likely to be an individual in the population.
  #stats = tools.Statistics(lambda ind: ind.fitness.values)
  #stats.register("avg", np.mean)
  #stats.register("std", np.std)
  #stats.register("min", np.min)
  #stats.register("max", np.max)

  #logbook = tools.Logbook()
  #logbook.header = ["gen", "nevals"] + stats.fields
  #-------------------------------------------------------------------------------
  #------------------------ Start of execution of each job -----------------------
  for job in range(n_jobs):
      #---------------------------------------------------------------------------
      # Start of time measurement
      # start_time  # Timing not directly translated, can use time.time() if needed
      start_time = time.time()
      # Discounts the initial node connection time in the calculation (only 1 time for each job)
      aux = jl[job] - t_c
      #------------- Printing job attribute values ​​to the screen -----------------
      print("\n---------------------------------------------------------")
      print(f"\n Processing the Job {job}:")
      print(f"\n Job {job}[{jr[job]}, {jb[job]}, {jl[job]}, {jo[job]}] waiting...")
      print("\n---------------------------------------------------------\n")
      #---------------------------------------------------------------------------
      # ------------------------ Variables initialization ------------------------
      # Output value of the population generated by NSGA3
      population = []
      # Output value of the population generated by NSGA3, without replication in the solutions
      final_population_no_replicate = []
      # Get unique rows based on unique values' indexes
      final_population = []
      #---------------------------------------------------------------------------
      # --------------------------------------------------------------------------
      # Initializing population
      population = toolbox.population(n=population_size)
      population = [toolbox.individual() for _ in range(population_size)]
      #---------------------------------------------------------------------------
      #-------------------- Nodes allocated to each job --------------------------
      # Stores the number of nodes allocated in the network, in the best solution, for the execution of a given job.
      cont_nos = 0
      # Separately stores the indexes of the nodes that make up the solution for each job
      v_nodes = []
      # --------------------------------------------------------------------------
      # Stores the position of the minimum solution (best solution)
      better_pos = 0
      # Nº of feasible solutions final_population_no_replicate
      num_solutions = 0
      # Nº of feasible solutions final_population (sem replicação)
      n_sol_feasible = 0
      # Stores the individual with the smallest value of OF
      best_sol_ind = []
      # Stores the value of the best solution
      best_sol = 0
      # Index of vectors NSS_R and NSS_B that stores the position of the minimum solution
      # List of solution values
      solutions_val = []
      # Vectors used to store the values ​​of objective functions and their positions
      v_sol_f1 = []  # f1 (Resources)
      v_sol_f2 = []  # f2 (Bandwidth)
      v_sol_f3 = []  # f3 (Latency)
      v_sol_fx = []  # Stores the value of the objective function: fx=f1^2+f2^2-f3
      v_sol_ind = []  # Stores the individuals of feasible solutions (binary tuples)
      # ---------------------- Initializing Search Spaces ------------------------
      # List of lists containing the search space of Latency of all nodes in the layer
      N_L = np.array([])
      #--- Stores latencies along a specific path, starting from a source node----
      N_L = np.array(AN_L[jo[job]])  # Selects only 1 row from the latency matrix
      #print('N_L List (One path):')
      #print(N_L)
      #input('ENTER...')
      #---------------------------------------------------------------------------
      #------------------------------------------------- Register the evaluate function and call the partial function with the parameters of a given job ---------------------------------------------
      # Pass the objectiveFunction function directly to toolbox.register, and use partial to pre-configure the function parameters.
      # Registering the objective function in the toolbox
      toolbox.register("evaluate", partial(objectiveFunction, aux=aux, job=job, jr=jr, jb=jb, N_R=N_R, N_B=N_B, N_L=N_L, v_sol_f1=v_sol_f1, v_sol_f2=v_sol_f2, v_sol_f3=v_sol_f3, v_sol_fx=v_sol_fx, v_sol_ind=v_sol_ind, num=numNodes))
      # Decorating the evaluation function, using the lambda function that checks whether an individual (ind) satisfies the constraints.
      # The penalty function to return a fitness value (INF) for any constraint violation, ensuring that those individuals will not be selected
      toolbox.decorate("evaluate", tools.DeltaPenality(lambda ind: jobConstraints(ind, aux, job, jr, jb, N_R, N_B, N_L, numNodes), INF))
      #------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
      # ------------------------------------------------- NSGA-III function call ----------------------------------------------------------------------------------------------------------------------
      #----------------------------------- Register the evaluate function and call the partial function with the parameters of a given job ------------------------------------------------------------
      # evaluate - calculates the fitness of an individual or a solution.
      # It is making use of statistics (stats=stats) or (stats=None)
      #logbook =
      algorithms.eaMuPlusLambda(population, toolbox, mu=population_size, lambda_=population_size, cxpb=cx_prob, mutpb=mut_prob, ngen=generations, stats=None, halloffame=None, verbose=True)
      #-----------------------------------------------------------------------------------------------------------------------------------
      # Obtaining the values ​​of the objective functions (R, B and L)
      #print("Logbook History:")
      #for record in logbook:
      #   print(record)
      #---------------------------------------------------------------------------
      # Stores the value of final_population without replication for printing on the graph
      final_population_no_replicate = np.array(list(zip(v_sol_fx, v_sol_ind, v_sol_f1, v_sol_f2, v_sol_f3)), dtype=object)
      # --------------------------------------------------------------------------
      # Accessing values ​​correctly
      values = [t[0] for t in final_population_no_replicate]
      individuals = [t[1] for t in final_population_no_replicate]

      # Convert individuals to a tuple for processing
      # It may happen that different tuples produce the same value in the OF
      #individuals_as_tuples = [tuple(individual) for individual in individuals]

      # Finding unique indexes using a set (individuals variable)
      seen = set()
      unique_indices = []
      for i, indiv in enumerate(values):
      #for i, indiv in enumerate(individuals_as_tuples):
          if indiv not in seen:
              seen.add(indiv)
              unique_indices.append(i)

      # Get unique rows based on unique values' indices
      final_population = np.array(final_population_no_replicate[unique_indices])
      # Sorted final_population
      final_population = sorted(final_population, key=lambda x: x[0])
      final_population = np.array(final_population)  # Ensure it's a NumPy array
      size_final_population = len(final_population)
      #print(f"size_final_population: {size_final_population}")
      #for i in range(10):
       #individual = final_population[i]
       #print(f"\nfinal_population_no_replicate: {final_population_no_replicate[i]}")
       #print(f"\nfinal_population: {final_population[i]}")
       #print(f"\nOF: {individual[0]}")
       #print(f"\ntupla: {individual[1]}")
       #print(f"\ndiff_X: {individual[2]}")
       #print(f"\ndiff_Y: {individual[3]}")
       #print(f"\ndiff_Z: {individual[4]}")
      #input("ENTER...")

      #---------------------------------------------------------------------------
      if size_final_population> 0: # If final_population is not empty
          #---------------------------------------------------------------------------
          # Find the individual with the smallest OF value (value of first the best solution)
          best_sol_ind = min(final_population, key=lambda x: x[0])
          # Find the index position of the smallest value
          best_sol = best_sol_ind[0]
          # better_pos = final_population do Array np
          # final_population[:, 0]: Selects the first column of final_population.
          # best_sol[0]: Accesses the first value of best_sol to compare with the first column of final_population.
          # np.where(...)[0]: Gets the indices (rows) where the condition is met.
          # [0]: Gets the first index from this list of indices, which will be the first place where the condition is true.
          better_pos = np.where(final_population[:, 0] == best_sol_ind[0])[0][0]
          #---------------------------------------------------------------------------
          # Select the individuals in the best position
          individuals = final_population[better_pos, 1]
          size_final_population = len(final_population)
          #print(f"individuals: {individuals}")
          #print(f"final_population.size: {final_population.size}")
          # Transfer remaining resources to attributes of last allocated node
          # Returns number of nodes in best solution - variable cont_nos
          job_R = jr[job]  # job_R: stores the remaining Resources (R) for jobs that will be allocated
          job_B = jb[job]  # job_B: stores the remaining Bandwidths (B) for jobs that will be allocated
          # job_L = jl[job]  # job_L: stores the remaining Latencies (L) for jobs that will be allocated
          cont = 0 # No. of individuals in final_population (stop condition)
          # While R and B are >=0 and count is less than the size of final_population
          while ((job_R > 0 or job_B > 0) and (size_final_population > cont)):
              t_sol = False # True if a solution has been found
              num_ones_individual = 0 # Number of ones in an individual
              cont_nodes_not_busy = 0 # Number of nodes that are not busy in an individual
              # Counts the number of ones (1s) in an individual
              num_ones_individual = sum(1 for i in individuals if i == 1)
              #print(f"Número de uns: {num_ones_individual}")

              # Check the number of individuals that are not in the state V_Busy  = 1
              for k in range(numNodes):
                if (individuals[k] == 1) and (V_Busy[k] == 0):
                  cont_nodes_not_busy += 1

              #print(f"cont_nodes_not_busy: {cont_nodes_not_busy}")

              # Stores the nodes allocated in the network, in the best solution found, for the execution of a specific job
              if (num_ones_individual == cont_nodes_not_busy):
                  for j in range(numNodes):
                    if (individuals[j] == 1) and (V_Busy[j] == 0):
                        v_nodes.append(j)
                        cont_nos += 1

                  # Updating the indexes of network nodes that are allocated (busy)
                  size_v_nodes = len(v_nodes)
                  if (size_v_nodes > 0):

                    for i in range(cont_nos):
                        pos_element = v_nodes[i]  # Returns correct position to be changed in lists: N_R and N_B
                        if N_R[pos_element] >= job_R:
                          N_R[pos_element] -= job_R
                          job_R = 0
                        else:  # Returns remaining values of last node
                          job_R -= N_R[pos_element]
                          N_R[pos_element] = 0

                        if N_B[pos_element] >= job_B:
                          N_B[pos_element] -= job_B
                          job_B = 0
                        else:  # Returns remaining values of last node
                            job_B -= N_B[pos_element]
                            N_B[pos_element] = 0  # Resets the node's bandwidth

                    if (job_R == 0 and job_B == 0):  # A solution has been found
                        t_sol = True
                        print(f"\nNode(s) allocated: {v_nodes}")
                        print(f"\naux: {aux}")
                        break  # Found a solution (exit the while loop)
                    else:
                      # Find the next minimum in the ordered sequence
                      # From the second onwards, these are the next minimums
                      cont += 1  # Updating the individual counter
                      if (size_final_population > cont):
                        next_min = final_population[cont]  # The next best solution
                        #print(f"\nThe next best solution (cont={cont}): {next_min}")
                        # Store the value of OF for the next best solution
                        best_sol = next_min[0]
                        # Finding the position of the next minimum
                        better_pos = np.where(final_population[:, 0] == next_min[0])[0][0]
                        #print(f"better_pos if(size_v_nodes): {better_pos}")
                        # Selecting the best positioned individuals
                        individuals = final_population[better_pos, 1]
                        #print(f"individuals if(size_v_nodes): {individuals}")
                        # Returns number of nodes in best solution - variable cont_nos
                        job_R = jr[job]  # job_R: stores the remaining Resources (R) for jobs that will be allocated
                        job_B = jb[job]  # job_B: stores the remaining Bandwidths (B) for jobs that will be allocated
                        # job_L = jl[job]  # job_L: stores the remaining Latencies (L) for jobs that will be allocated
                      else:
                        best_sol =  0  # When there is no solution
                        break  # Find the next minimum and exit the loop (while)
                      #---------------------------------------------------------------------------
                    #-----------------------if (job_R == 0 and job_B == 0)-------------------------

              else: # if (num_ones_individual == cont_nodes_not_busy):

                t_sol = False
                # Find the next minimum in the ordered sequence
                # From the second onwards, these are the next minimums
                cont += 1  # Updating the individual counter
                if (size_final_population > cont):
                    next_min = final_population[cont]  # The next best solution
                    #print(f"\nThe next best solution (cont={cont}): {next_min}")
                    # Store the value of OF for the next best solution
                    best_sol = next_min[0]
                    # Finding the position of the next minimum
                    better_pos = np.where(final_population[:, 0] == next_min[0])[0][0]
                    #print(f"better_pos if(size_v_nodes): {better_pos}")
                    # Selecting the best positioned individuals
                    individuals = final_population[better_pos, 1]
                    #print(f"individuals if(size_v_nodes): {individuals}")
                    # Returns number of nodes in best solution - variable cont_nos
                    job_R = jr[job]  # job_R: stores the remaining Resources (R) for jobs that will be allocated
                    job_B = jb[job]  # job_B: stores the remaining Bandwidths (B) for jobs that will be allocated
                    # job_L = jl[job]  # job_L: stores the remaining Latencies (L) for jobs that will be allocated
                else:
                    best_sol =  0  # When there is no solution
                    break  # Find the next minimum and exit the loop (while)
          #---------------------------------------------------------- End while ----------------------------------------------------------------------------------

          if t_sol: # if t_sol = True:
            # The IoT network nodes that are part of the solution are allocated
            for i in range(size_v_nodes):
                    v_nodes_network.append(v_nodes[i])
                    print(f"\nNode(s) Network allocated: {v_nodes_network}")
                    cont_nos_rede += 1

            # Mark solution nodes as busy
            V_Busy = mark_busy(cont_nos_rede, v_nodes_network, V_Busy)

            # -------------------------------------------------------------------------

            # Combine the three vectors into a solution matrix to generate the graphical solutions
            # solutions_val = np.array([v_sol_f1, v_sol_f2, v_sol_f3]).T  # Each column represents an objective
            solutions_val = np.array([final_population[:, 2], final_population[:, 3], final_population[:, 4]]).T  # Each column represents an objective
            #print(f"solutions_val: {solutions_val}")
            #input("ENTER...")
            # Total number of solutions of the final_population_no_replicate vector
            num_solutions = len(final_population)  # Nº. of solutions found
            #np.ones() creates an array of size num_solutions filled with the value 1.
            #dtype=bool converts these values ​​to True, since in a boolean context, 1 equates to True.
            pareto_front = np.ones(num_solutions, dtype=bool)

            # Check each solution against all others to see if it is dominated
            for i in range(num_solutions):
                for j in range(num_solutions):
                    if i != j and dominates(solutions_val[j][:], solutions_val[i, :]):
                        pareto_front[i] = False  # If solution j dominates solution i, then i is dominated
                        break  # If one solution is dominated, it can break the inner loop
            # Non-dominated solutions (Pareto front)
            pareto_solutions = solutions_val[pareto_front, :]

            # Dominated solutions
            dominated_solutions = solutions_val[~pareto_front, :]

            _FileName = os.path.basename(path_input).split('.')[0]
            save_path = f'{path_output}/results_{str(r)}_{_FileName}_job{job}.png'

            # Create a 3D graph to visualize solutions
            #fig = plt.figure('Job')
            # Increase chart size and resolution
            fig = plt.figure(figsize=(15, 15), dpi=100)  # Figure size and resolution
            ax = fig.add_subplot(111, projection='3d')  # Set ax before conditions

            # Check if there are dominated solutions to plot
            if dominated_solutions.size > 0:
                ax.scatter(dominated_solutions[:, 0], dominated_solutions[:, 1], dominated_solutions[:, 2], c='r', marker='o', label='Dominated solutions')  # Red for dominated solutions

            # Check if there are Pareto front solutions to plot
            if pareto_solutions.size > 0:
              ax.scatter(pareto_solutions[:, 0], pareto_solutions[:, 1], pareto_solutions[:, 2], c='b', marker='o', label='Pareto Front')  # Blue for non-dominated solutions (Pareto Front)
              # Print the best solution (in green)
              ax.scatter(solutions_val[better_pos][0], solutions_val[better_pos][1], solutions_val[better_pos][2], c='g', marker='o', s=100, label='Best solution')  # 's' controls the size of the green circle.

            # Add labels to axes
            ax.set_xlabel('F1 (Resource)')
            ax.set_ylabel('F2 (Bandwidth)')
            ax.set_zlabel('F3 (Latency)')

            # Add title to chart
            ax.set_title(f'Job {job} [{jr[job]}, {jb[job]}, {jl[job]}, {jo[job]}]')

            # Add the legend
            ax.legend()

            plt.savefig(save_path)

            # Pause to allow the graph to render, time (s)
            #plt.pause(0.01)

            # Close the figure to avoid displaying it on screen
            plt.close()

            # -------------------------------------------------------------------------
            # Total number of feasible solutions of final_population (without duplicates)
            n_sol_feasible = len(final_population)  # Nº. of solutions found, no replication
            #--------------------------------------------------------------------------
            # -------- Printing data to the output file: results.txt ---------
            # Prints the values ​​of the best solution in the results.txt file
            fileID.write("\n----------------------------------------------")
            fileID.write(f"\n Job {job}[{jr[job]}, {jb[job]}, {jl[job]}, {jo[job]}]")
            fileID.write("\n----------------------------------------------")
            fileID.write(f"\nNº of feasible solutions:  {n_sol_feasible}")
            fileID.write("\n----------------------------------------------")
            fileID.write(f"\nJob Resource (R): {jr[job]}")
            fileID.write(f"\nJob Bandwidth (B): {jb[job]}")
            fileID.write(f"\nJob Latency (L): {jl[job]}")
            fileID.write(f"\nJob source node (jo): {jo[job]}")
            fileID.write(f"\nObjective Function (OF): {best_sol:.1f}")
            fileID.write(f"\nJob connection time (ms): {t_c}")
            fileID.write(f"\nNode(s) allocated: {v_nodes}")
            # Calculates the total value of OF for all jobs
            OF_total += best_sol
          else:  # if t_sol = False:
            # -------- Printing data to the output file: results.txt ---------
            # final_population = 0
            # There are no feasible solutions
            n_sol_feasible = 0
            # resets the value of best_sol (OF)
            best_sol = 0 # Does not print the value of OF
            fileID.write("\n----------------------------------------------")
            fileID.write(f"\n Job {job}[{jr[job]}, {jb[job]}, {jl[job]}, {jo[job]}]")
            fileID.write("\n----------------------------------------------")
            fileID.write(f"\nNº of feasible solutions:  {n_sol_feasible}")
            fileID.write("\n----------------------------------------------")
            fileID.write(f"\nJob Resource (R): {jr[job]}")
            fileID.write(f"\nJob Bandwidth (B): {jb[job]}")
            fileID.write(f"\nJob Latency (L): {jl[job]}")
            fileID.write(f"\nJob source node (jo): {jo[job]}")
            fileID.write(f"\nObjective Function (OF):  ")
            fileID.write(f"\nJob connection time (ms): {t_c}")

      # End of time measurement
      end_time = time.time() - start_time
      total_time += end_time
      #print(f'\nElapsed time: {end_time:.2f} seconds')
      fileID.write(f'\nElapsed time: {end_time:.2f} seconds')
      fileID.write("\n----------------------------------------------")

  # Allocated nodes of the entire network
  #print(f"\nAllocated nodes in the network (Original seq.): {v_nodes_network}")
  fileID.write(f"\nAllocated nodes in the network (Original seq.): {v_nodes_network}")
  # Sorts (ascending) the allocated nodes of the network
  v_nodes_network_order = sorted(v_nodes_network)
  #print(f"\nAllocated node(s) in the network (Ordered seq.): {' '.join(map(str, [no for no in v_nodes_network_order]))}")
  fileID.write(f"\nAllocated node(s) in the network (Ordered seq.): {' '.join(map(str, [no for no in v_nodes_network_order]))}\n")

  print('-------------------------------------------------------------')
  print(f'The results are in the file: {name_file}')
  print('-------------------------------------------------------------')
  #print(f"\nV_Busy: {V_Busy}")
  #print(f"\nTotal OF: {OF_total:.2f}")
  #print("-------------------------------------------------------------")
  fileID.write("\n-------------------------------------------------------------")
  fileID.write(f"\nTotal OF: {OF_total:.1f}")
  fileID.write("\n-------------------------------------------------------------")
  fileID.write(f"\nTotal time: {total_time:.2f} seconds")
  fileID.write("\n-------------------------------------------------------------")
  #print(f"Total time: {total_time:.2f} seconds")
  #print("-------------------------------------------------------------\n")
  #--------------------------------------------------------------------------------
  # Final call to keep all chart windows open
  # plt.show()  # Blocking end call to keep graphs open
  #--------------------------------------------------------------------------------
  # Close the file: results.txt
  fileID.close()

#drive.mount('/content/drive')

# Set the directory path where the .json files are located

# Setting default values
default_in_f = r'C:\Users\Murilo\Documents\DOUTORADO\UFMS-FACOM\Material-Ricardo\Encontros-Ricardo\Encontro_41-31-10-2024\python\23nds\NSGA3\input'
default_out_f = r'C:\Users\Murilo\Documents\DOUTORADO\UFMS-FACOM\Material-Ricardo\Encontros-Ricardo\Encontro_41-31-10-2024\python\23nds\NSGA3\output'

# Prompts user for directory path or uses default value if ENTER is pressed
input_folder = input(f"Enter an input directory path or press ENTER for default (ex: {default_in_f}): ") or default_in_f

output_folder = input(f"Enter an output directory path or press ENTER for default (ex: {default_out_f}): ") or default_out_f

# Setting default values
default_runnings = 1

# Number of runnings of a given configuration
numRunnings = get_positive_integer(f"Number of runnings: ", default_runnings)

# Create the output folder if it doesn't exist; keep if it already exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all .json files in input directory
json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]

# Process each .json file and generate a .txt result file
for json_file in json_files:
    # Full path of input file
    input_path = os.path.join(input_folder, json_file)

    main(input_path, output_folder, numRunnings)